# Tutorial: 3D Visualization
In this tutorial we demonstrate the use of the 3D Visualization capabilities in pumapy. The 3D visualization is based on a coupling with paraview and VTK. Visualization can be performed on either raw tomography samples (each material phase is composed of a range of grayscale values) or segmented images (each material phase is a single grayscale value). Pumapy can currently produce volume renderings or contours. Examples of each visualization are shown below. 

First, we must import puma:

In [1]:
import os

# Note - "conda activate puma" must be run prior to calling pumapy
import pumapy as puma

# Note - the first time puma is run on a machine, the import command will take a couple minutes as the cython code has to be compiled

Next we must either import or generate a material to visualize. Computational domains are stored in the workspace class. Below we show an example of both importing and computationally generating a material:

In [ ]:
# Generating a workspace of randomly placed, intersecting sphers: 
#.    size of domain in voxels: (200,200,200)
#.    diameter of each sphere in voxels: 20
#.    porosity of generated material: 0.7
ws_generated = puma.generate_random_spheres((200,200,200), 20, 0.7)

# The voxel length (in meters) of the workspace defaults to 1 micron (1e-6 m).
# To change the voxel length, modify the parameter directly: 
ws_generated.voxel_length = 1.3e-6

# Next we will import an example tomography file of size 200^3 and voxel length 1.3e-6
ws_imported = puma.import_3Dtiff("python/tests/testdata/200_FiberForm.tif", 1.3e-6)


The workspaces above have not been segmented yet. In the case of generated workspace, the generated material has been stored for a grayscale [128,255] and the void phase is stored between [0,127]. The appropriate grayscale cutoff for the imported tomography sample is 90, such that [90,255] is material and [0,89] is the void. These values will be different for each tomography image. 

First, we will generate both volume and contour renderings of the two materials without segmentation. By default, the contour triangles are a solid color. If you would prefer the colors to be weighted by the local grayscale value, the color option can be set to true in the visualization:

In [ ]:
# Generating Volume and Contour renderings of the computationally generated material
#    The grayscale range of the material to be rendered is specified as (128,255) and is inclusive
puma.render_contour(ws_generated, (128,255))
puma.render_contour(ws_imported, (90,255))

puma.render_volume(ws_generated, (128,255))
puma.render_volume(ws_imported, (90,255))

# Note - there is a bug with VTK and MacOS where the 3D volume is sometimes unresponsive when first opened.
# If this occurs, minimize and then expand the window, and it should work properly

Next, we will segment the workspaces, such that all void voxels contain an ID of 0 and all solid voxels contain an ID of 1. This could be expanded for multi-phase domains. 

In [ ]:
# Setting all grayscale values between 0 and 127, inclusive, to 0
ws_generated.set_material_id((0,127), 0)
# Setting all grayscale values between 128 and 255, inclusive, to 1
ws_generated.set_material_id((128,255),1)

# Setting all grayscale values between 0 and 89 to 0. This represents the void phase in the tomography file
ws_imported.set_material_id((0,89), 0)
# Setting all grayscale values between 90 and 255 to 1. This represents the solid phase in the tomography file
ws_imported.set_material_id((90,255),1)


Now we will generate contour and volume renderings of the segmented files. Note that the grayscale values have changed. In the case of the contour render, the new grayscale values can be specified by either setting the appropriate range - (1,1) in this case, or by specifying the material ID value itself - 1 in this case. For the volume render, the range of grayscales to be considered must be included, in this case simply (1,1)

In [ ]:
puma.render_contour(ws_generated, 1)
puma.render_contour(ws_imported, 1)

puma.render_volume(ws_generated, (1,1))
puma.render_volume(ws_imported, (1,1))

Note that the contour renders for the segmented images are significantly less smooth than for the non-segmented images. This is because for segmented images, the triangulation algorithms have significantly less degrees of freedom when assigning triangle angles, resulting in a rougher surface than for non-segmented images. 